# 13.0 Dynamic Programming

## 13.1 Ways to climb a staircase

### Problem Statement
Implement a function that given some height of a staircase, named $h$, and set of possible steps, named $S$, returns all of the unique ways to climb the staircase.

In [1]:
from collections import namedtuple
import unittest


def staircase(h, S, cache={}):
    """Return unique ways to climb staircase height h using steps S."""

    soltns = set()
    for s in S:
        if s == h:
            soltns.add((s,))
        elif h-s > 0:
            subs = (staircase(h-s, S, cache=cache) 
                    if h-s not in cache else cache[h-s])
            for sub in subs:
                soltns.add((s,) + sub)
                soltns.add(sub + (s,))

    cache[h] = soltns  # Update cache.
    return soltns


class StaircaseTest(unittest.TestCase):

    def test_staircase(self):
        case = namedtuple('case', ['h','S','expected'])
        cases = [
            case(2, {1}, {(1,1)}),
            case(2, {1,2}, {(1,1),(2,)}),
            case(3, {1}, {(1,1,1)}),
            case(3, {1,2}, {(1,1,1),(1,2),(2,1)}),
            case(3, {1,2,3}, {(1,1,1),(1,2),(2,1),(3,)}),
            case(4, {1,2}, 
                 {(1,1,1,1),(2,1,1),(1,2,1),(1,1,2),(2,2)}),
            case(4, {1,2,3}, 
                 {(1,1,1,1),(2,1,1),(1,2,1),(1,1,2),(2,2),(1,3),(3,1)}),
            case(5, {1,2,3},
                 {(1,1,1,1,1),
                  (2,1,1,1),(1,2,1,1),(1,1,2,1),(1,1,1,2),
                  (2,2,1),(1,2,2),(2,1,2),
                  (3,1,1),(1,3,1),(1,1,3),(3,2),(2,3)}),
        ]
        for c in cases:
            rcv = staircase(c.h, c.S, cache={})
            self.assertEqual(rcv, c.expected)


unittest.main(StaircaseTest(), argv=[''], verbosity=2, exit=False)

test_staircase (__main__.StaircaseTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## 13.2 Ways to decode a string

### Problem Statement
Given a mapping of characters to number such as $a=1, b=2, \cdots, z=26$ and a message encoded using this mapping, return all of the unique ways the message can be decoded.

Only messages with valid decodings are passed as input.

#### Examples
```
message  : 111
decodings: {'aaa','ak','ka'}
```

In [2]:
from collections import namedtuple
import unittest


# ALPHA maps an integer to a character in the alphabet used by messages.
ALPHA = {str(ind+1):chr(num) 
         for ind, num in enumerate(range(ord('a'), ord('z')+1))}


def cartesian_product_str(strset1, strset2):
    """Return the cartesian product of sets of strings."""
    product = set()
    for str1 in strset1:
        for str2 in strset2:
            product.add(''.join([str1, str2]))
    return product


def decoder(msg, alpha=ALPHA, cache={}):
    """Return unique ways to decode msg using alpha."""

    # Initialize solution to either empty set or alphabet value.
    # When the full message is in the alphabet then the only
    # other subset of the full message is the empty set.
    soltns = set() if msg not in alpha else set([alpha[msg]])

    # Split the message into 3 parts: left, center, right
    # Splitting simplifies first and last character handling.
    # All messages decoded within the loop are composites of
    # characters from the alphabet.  As a result, the solution
    # requires taking cartesian product of composite solutions.
    for ind, c in enumerate(msg):
        subs = set([alpha[c]])
        if ind > 0:
            lmsg = msg[:ind]
            lsoltn = (decoder(lmsg, alpha, cache=cache) 
                      if lmsg not in cache else cache[lmsg])
            subs = cartesian_product_str(lsoltn, subs)
        if ind+1 < len(msg):
            rmsg = msg[ind+1:]
            rsoltn = (decoder(rmsg, alpha, cache=cache)
                      if rmsg not in cache else cache[rmsg])
            subs = cartesian_product_str(subs, rsoltn)
        soltns = soltns.union(subs)

    cache[msg] = soltns  # Update cache.
    return soltns


class DecoderTest(unittest.TestCase):
    
    def test_decoder(self):
        case = namedtuple('case', ['msg', 'expected'])
        cases = [
            case('1', {'a'}),
            case('12', {'ab','l'}),
            case('123', {'abc','aw','lc'}),
            case('1234', {'abcd','awd','lcd'}),
            case('111', {'ka','ak','aaa'}),
        ]
        for c in cases:
            rcv = decoder(c.msg, cache={})
            self.assertEqual(rcv, c.expected)


unittest.main(DecoderTest(), argv=[''], verbosity=2, exit=False)

test_decoder (__main__.DecoderTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK
